In [2]:
### Solve Heat Equation

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import numpy.testing as nptest
import scipy.sparse as sparse
import scipy.sparse.linalg as sparselinalg

In [3]:
# we are trying to solve problem for which u(x,y) = x**4*y**5-17*sin(xy) is a solution

In [4]:
def u(x,y): return x**4*y**5-17*np.sin(x*y)

In [5]:
#RHS
def f(x,y): return -(12*x**2*y**5+20*x**4*y**3+(x**2+y**2)*17*np.sin(x*y))

In [6]:
# our Omega is (0,1)x(0,1)
###generate RHS f, order nodes lexicographically
# determine pointwise values for u



In [7]:
#generate mesh, i.e. x_i,y_i (including the boundary)
# our n ranges from 2 to 8, i.e. if n=2, we have h_x=h_y=1/4
# if h=2**(-n), then there are 2**n+1 grid points in each direction

def meshvector(n):
    h=2**(-n)
    pts=2**n-1
    x=np.zeros(((2**n-1)**2,2))
    # print(len(x))
    for j in range(pts):
        for i in range(pts):
            x[pts*j+i,0]=(i+1)*2**(-n)
            x[pts*j+i,1]=(j+1)*2**(-n)
            # print((i,j))
            # print(x[n*j+i,:])
    return x

In [8]:
# determine values for f on mesh

def f_on_mesh(x,n):
    if(n==1):
        print("error, n too small")
    h=2**(-n)
    #print(h)
    #print(1/h**2)
    len_x=len(x)
    f_mesh = np.zeros(len_x)
    f_meshNoBound = np.zeros(len_x)
    for i in range(len_x):
        #print("index", i)
        f_mesh[i]=f(x[i,0],x[i,1])
        f_meshNoBound[i]=f_mesh[i]
        #print(f_mesh[i])
        if(i<2**n-1):
            f_mesh[i]+=u(x[i,0],x[i,1]-h)/h**2
            #print(1)
        if(i>=(2**n-1)*(2**n-2)):
            f_mesh[i]+=u(x[i,0],x[i,1]+h)/h**2
            #print(2)
        if((i % (2**n-1))==(2**n-2)):
            f_mesh[i]+=u(x[i,0]+h,x[i,1])/h**2
            #print(3)
            #print(u(x[i,0]+h,x[i,1])/h**2)
            #print(u(x[i,0]+h,x[i,1]))
        if((i % (2**n-1))==0):
            f_mesh[i]+=u(x[i,0]-h,x[i,1])/h**2
            # print(i % (2**n-1))
            #print(4)
            # print(x[i,:])
            # print(f_mesh[i])
    # print("f_mesh_no_bound: ", f_meshNoBound)
    return f_mesh        

In [9]:
# generating B part of A

def BpartA(n):
    h=1/2**n
    N=2**n-1
    data=[[-1/h**2]*(N)**2, [4/h**2]*(N)**2, [-1/h**2]*(N)**2]
    offsets = np.array([-1,0,1])
    b = sparse.dia_matrix((data, offsets),shape=(N,N)).toarray() 
    bmultiple = (b,)*(N)
    return sparse.csc_matrix(sparse.block_diag(bmultiple))

In [10]:
def CpartA(n):
    h=1/2**n
    N=2**n-1
    cdata=[[-1/h**2]*(N)**2,[-1/h**2]*(N)**2]
    offset=np.array([N,-N])
    return sparse.csc_matrix(sparse.dia_matrix((cdata, offset),shape=((N)*(N),(N)*(N))))

In [11]:
def generateA(n):
    A=BpartA(n)+CpartA(n)
    #print(A.shape)
    return A

In [12]:
#compute the real solution at the grid points to check our results
for n in range(2,10):
    x=meshvector(n)
    u_real=u(x[:,0],x[:,1])
    u_approx=sparselinalg.spsolve(sparse.csc_matrix(generateA(n)), f_on_mesh(x,n))
    print("n= ",n, " Error = ", np.max(abs(u_real-u_approx)))

n=  2  Error =  0.00385316722582
n=  3  Error =  0.00112317320866
n=  4  Error =  0.000295968293221
n=  5  Error =  7.4670066736e-05
n=  6  Error =  1.87102695381e-05
n=  7  Error =  4.68136892984e-06
n=  8  Error =  1.17052483262e-06
n=  9  Error =  2.92654362966e-07
